In [ ]:
# Imports
import numpy as np
import pandas as pd
import yfinance as yf
import hvplot.pandas
import plotly.graph_objs as go
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path
import os
from datetime import datetime, timedelta
import pandas as pd
import mplfinance as mpf

In [ ]:
#pull data from yahoo finance
data =yf.download(tickers='COST', period='max', interval = '1d', start='2007-01-01')

In [ ]:
# Show Data
print(data)

In [ ]:
# build dataframe

cost_df = pd.DataFrame(data)
cost_df.head(10)

# change df to csv
cost_csv = cost_df.to_csv('/users/manu/desktop/project_2/cost_csv', index=True)

#Show the df
cost_df.head(10)
# cost_df['Costco']= cost_df['Close']

In [ ]:
#Show Close Only
Cost_close = cost_df.drop(columns =['Open','High','Low','Adj Close','Volume'])
Cost_close.head(10)

In [ ]:
Cost_returns = Cost_close.pct_change().dropna()
Cost_returns

In [ ]:
# Access to the News Api
newsapi = NewsApiClient(api_key='ce3e78ab07ed4649a677e4da44c06a82')

In [ ]:
# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()


In [ ]:
# Define a function to build news df
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

In [ ]:
# Pull all the Costco Stock headlines data from 2008 
Cost_news_en = newsapi.get_everything(
    q=" COST AND Costco AND 2008",
    language="en"
)
# Get first topic
# aapl_headlines, dates = get_headlines("apple")

# Show the total number of news
print(Cost_news_en["totalResults"])

In [ ]:
# Create a DataFrame with the Costco News
Cost_en_df = create_df(Cost_news_en["articles"], "en")
Cost_en_df

In [ ]:
# Create CSV from News df
Cost_en_df.to_csv('/users/manu/desktop/project_2/cost_csv', index=False, encoding='utf-8-sig')

In [ ]:
# Build a sentiment analysis df 

Cost_sentiment = []
analyzer = SentimentIntensityAnalyzer()

for article in Cost_news_en['articles']:
    try:
        date = article["publishedAt"][:10]
        text = article["content"][0:198]
        sentiment = analyzer.polarity_scores(text)
        pos = sentiment['pos']
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        compound = sentiment["compound"] 
        
        Cost_sentiment.append({
            "text": text,
            "date": date,
            "positive": pos,
            "neutral": neu,
            "negative": neg,
            "compound": compound 
        })
        
    except AttributeError:
        pass

Cost_S_df = pd.DataFrame(Cost_sentiment) 
cols = ['date', 'positive', 'neutral', 'negative', 'compound', 'text']
Cost_S_df = Cost_df[cols] 
Cost_S_df.describe()

In [ ]:
#Display a candle stick chart
fig = go.Figure(data=[go.Candlestick(x=cost_df.index,
                open=cost_df['Open'],
                high=cost_df['High'],
                low=cost_df['Low'],
                close=cost_df['Close'])])

fig.show()